In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN

%config InlineBackend.figure_format = 'retina'
%matplotlib inline
plt.rcParams['font.size'] = 12
sns.set(style="darkgrid")
plt.rcParams['savefig.format'] = 'pdf'

Загрузка данных

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_preprocessed (1).csv').drop(columns=['Unnamed: 0'])

In [ ]:
data.head()

,pharmacy,card_num,receipt_date,product,cashier_num,receipt_num,application,retail_price,receipt_unique,month
0,0,2363656,2018-01-05,784213,0,1,379517,530.74,2018-01-05 1 0,1
1,0,2363656,2018-01-05,784213,0,1,379517,530.74,2018-01-05 1 0,1
2,0,2364030,2018-01-04,768150,0,3,379753,79.31,2018-01-04 3 0,1
3,0,2364030,2018-01-04,778137,0,3,379753,217.95,2018-01-04 3 0,1
4,0,2364030,2018-01-15,772668,0,4,379956,745.22,2018-01-15 4 0,1


Получим данные для кластеризации

In [ ]:
def transform_data(data):
    dataset = data.groupby('card_num').agg({'receipt_unique': 'nunique', 
                                            'product': 'nunique', 
                                            'receipt_date': 'nunique',
                                            'application': 'nunique',
                                            'retail_price': ['mean', 'std', 'min', 'max']
                                            })
    
    # число позиций в чеке в месяц (среднее, минимальное, максимальное)
    dataset['mean_purch_month'] = data.groupby(['card_num', 'month']).size().unstack().fillna(0).mean(axis=1)
    dataset['min_purch_month'] = data.groupby(['card_num', 'month']).size().unstack().fillna(0).min(axis=1)
    dataset['max_purch_month'] = data.groupby(['card_num', 'month']).size().unstack().fillna(0).max(axis=1)

    # сумма покупок в месяц (средняя, минимальная, максимальная)
    dataset['mean_price_month'] = data.groupby(['card_num', 'month'])['retail_price'].mean().unstack().fillna(0).mean(axis=1)
    dataset['min_price_month'] = data.groupby(['card_num', 'month'])['retail_price'].mean().unstack().fillna(0).min(axis=1)
    dataset['max_price_month'] = data.groupby(['card_num', 'month'])['retail_price'].mean().unstack().fillna(0).max(axis=1)

    # число дней походов в аптеку в месяц (среднее и максимальное)
    dataset['mean_days_month'] = data.groupby(['card_num', 'month'])['receipt_date'].nunique().unstack().fillna(0).mean(axis=1)
    dataset['max_days_month'] = data.groupby(['card_num', 'month'])['receipt_date'].nunique().unstack().fillna(0).max(axis=1)

    # среднее число различных применений, купленных за месяц (одно и то же или куча разных) 
    dataset['mean_app_month'] = data.groupby(['card_num', 'month'])['application'].nunique().unstack().fillna(0).mean(axis=1)
    # число различных применений, купленных за последний месяц
    dataset['app_last_month'] = (
        data.groupby(['card_num', 'month'])['application']
        .nunique()
        .reset_index(level=1)
        .groupby('card_num')['application']
        .last()
    )
    
    # в конце заполним пропуски, тк стандартное отклонение может отсутствовать для покупателей, 
    # у которых в train попала лишь одна покупка
    return dataset.fillna(0)


In [ ]:
clust_dataset = transform_data(data)

In [ ]:
scaler = StandardScaler()

clust_scaled = scaler.fit_transform(clust_dataset)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


## Кластеризация

### K-Means

In [ ]:
n_clusters = 10

In [ ]:
clstr = KMeans(n_clusters=n_clusters, random_state=42)

clusters = clstr.fit_predict(clust_scaled)

In [ ]:
kmeans_clusters = pd.DataFrame(clusters, index=clust_dataset.index, columns=['cluster_num'])
kmeans_clusters.head()

,cluster_num
card_num,
2,9
4,9
6,2
8,2
10,9


In [ ]:
# kmeans_clusters.to_csv('kmeans_clusters.csv')

### DBSCAN

In [ ]:
clstr_d = DBSCAN(eps=1.1)

clusters_d = clstr_d.fit_predict(clust_scaled)

In [ ]:
len(set(clusters_d))

34

In [ ]:
dbscan_clusters = pd.DataFrame(clusters_d, index=clust_dataset.index, columns=['cluster_num'])
dbscan_clusters.head()

,cluster_num
card_num,
2,-1
4,-1
6,-1
8,-1
10,-1


Какая доля покупателей не соответствует ни одному кластеру:

In [ ]:
dbscan_clusters[dbscan_clusters['cluster_num'] == -1].shape[0] / dbscan_clusters.shape[0]

0.07943484064198977

In [ ]:
# dbscan_clusters.to_csv('dbscan_clusters.csv')